In [ ]:
# 安裝必要套件
!pip install langgraph openai


In [ ]:
# 導入模組
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI
from google.colab import userdata

# 定義狀態結構
@dataclass
class ReActState:
    input: str
    output: str = ""
    history: list = None
    iteration: int = 0

    def __post_init__(self):
        if self.history is None:
            self.history = []

# 初始化OpenAI客戶端
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 替換為您的API金鑰

# 語言模型調用函數
def llm_call(prompt: str, max_tokens: int = 100) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

# 生成節點：翻譯中文到英文
def generate_node(state: ReActState) -> Dict:
    prompt = f"將以下中文翻譯成英文，要求語義準確：\n{state.input}\n歷史：{state.history}"
    state.output = llm_call(prompt)
    state.history.append(f"生成：{state.output}")
    return {"output": state.output, "history": state.history}

# 檢查節點：更新迭代次數
def check_node(state: ReActState) -> Dict:
    state.iteration += 1
    return {"iteration": state.iteration}

# 構建工作流程
workflow = StateGraph(ReActState)
workflow.add_node("generate", generate_node)
workflow.add_node("check", check_node)
workflow.set_entry_point("generate")
workflow.add_edge("generate", "check")
workflow.add_conditional_edges(
    "check",
    lambda state: "end" if state.iteration >= 2 else "generate",
    {"generate": "generate", "end": END}
)
graph = workflow.compile()

# 執行範例
result = graph.invoke(ReActState(input="人工智慧改變未來"))
print(f"最終英文翻譯：{result['output']}")
print(f"翻譯歷史：{result['history']}")